In [ ]:
import numpy as np
import pandas as pd
import GPy
import sys
sys.path.append('../')
import BayesianOptimization

In [ ]:
# Mixed integer optimization problem
# Objective function
# f(x,y) = -(10*y1 + 15*y2 + 20*y3 + x1 + 1.5*x2 + 2*x3) - 5*x4 - 9.5*x5 - 0.5*x7 - 0.5*x8 + 25*x9 + 25*x10
# Constraints
# Equality constraints
# g1(x) = x5 + x6 - (x7 + x8)
# g2(x) = x6 - 0.9*x4
# g3(x) = x9 - 0.82*x7
# g4(x) = x10 - 0.95*x8
# Inequality constraints
# h1(x) = y2 + y3 - 1
# h2(x) = x1 - U*y1
# h3(x) = x2 - U*y2
# h4(x) = x3 - U*y3
# h5(x) = x4 - x1
# h6(x) = x7 - x2
# h7(x) = x8 - x3
# h8(x) = x9 + x10 - d
# U = 20, d = 10

# Optimal solution
# x = [11.695906433, 0.0, 10.526315789, 11.695906433, 0.0, 10.526315789, 0.0, 10.526315789, 0.0, 10.0]
# y = [1, 0, 1]

# Objective function
def f(x, y):
    return -(10*y[:,0] + 15*y[:,1] + 20*y[:,2] + x[:,0] + 1.5*x[:,1] + 2*x[:,2]) - 5*x[:,3] - 9.5*x[:,4] - 0.5*x[:,6] - 0.5*x[:,7] + 25*x[:,8] + 25*x[:,9]

# Equality constraints
def g(x, y):
    G = np.zeros((x.shape[0], r))
    G[:,0] = x[:,4] + x[:,5] - (x[:,6] + x[:,7])
    G[:,1] = x[:,5] - 0.9*x[:,3]
    G[:,2] = x[:,8] - 0.82*x[:,6]
    G[:,3] = x[:,9] - 0.95*x[:,7]
    return G

# Inequality constraints
def h(x, y):
    H = np.zeros((x.shape[0], s))
    H[:,0] = y[:,1] + y[:,2] - 1
    H[:,1] = x[:,0] - 30*y[:,0]
    H[:,2] = x[:,1] - 30*y[:,1]
    H[:,3] = x[:,2] - 30*y[:,2]
    H[:,4] = x[:,3] - x[:,0]
    H[:,5] = x[:,6] - x[:,1]
    H[:,6] = x[:,7] - x[:,2]
    H[:,7] = x[:,8] + x[:,9] - 10
    return H

# Number of continuous variables (n)
n = 10
# Number of discrete variables (m)
m = 3
# Total number of variables (n+m)
n_tot = n + m
# Number of equality constraints (r)
r = 4
# Number of inequality constraints (s)
s = 8
# Bounds

x = [11.695906433, 0.0, 10.526315789, 11.695906433, 0.0, 10.526315789, 0.0, 10.526315789, 0.0, 10.0]
x_l = np.zeros(n)
x_u = np.array([12, 0.1, 12, 12, 0.1, 12, 0.1, 12, 0.1, 12])
y_l = np.zeros(m, dtype=int)
y_u = np.ones(m, dtype=int)
# Bounds
bounds_x = np.array([x_l, x_u])
bounds_y = np.array([y_l, y_u])

In [ ]:
# Load initial data
D_0 = np.loadtxt("initial_data_ex3.csv", delimiter=",")

# Select surrogate model, AF, kernel and sampling strategy
surrogate = 'SGP'
kernel = GPy.kern.Matern52(input_dim=n_tot, lengthscale=1.0)
adquisition = "EI"
sample_y = "Bernoulli"
sample_x = "Gaussian"
param_est_Bernoulli = "Bayes"
param_est_Gaussian = "Bayes"

In [ ]:
labels_D = ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8","x9", "x10", "y1", "y2", "y3", "Omega"]
labels_D_best = ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8","x9", "x10", "y1", "y2", "y3", "Omega", "p", "f"]
points = 1000000
for i in range(5):
    # Run the optimization 10 times, save the solutions with tuples
    bo = BayesianOptimization.BO_GPy(f, g, h, None, None, None,
                                    bounds_x, bounds_y, kernel, n, m, 
                                    points = points, t_max = 500, j_max = 100,
                                    surrogate = surrogate, adquisition = adquisition, 
                                    sample_y = sample_y, sample_x = sample_x, param_est_bernoulli = param_est_Bernoulli, param_est_gaussian = param_est_Gaussian,
                                    D_0 = D_0, early_stop=False, DomainReduction=False, verbose=True)
    D, D_best = bo.fit()
    df_D = pd.DataFrame(D, columns=labels_D)
    df_D_best = pd.DataFrame(D_best, columns=labels_D_best)
    df_D.to_csv("Example_3_final" + ".csv", index=False, mode="a")
    df_D_best.to_csv("Example_3_best_final" + ".csv", index=False, mode="a")